In [1]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import napari
from tifffile import imsave
import skimage.measure
import time
import seaborn as sns
import matplotlib.pyplot as plt
import cc3d
from progressbar import ProgressBar

C:\Users\Gast-User\.conda\envs\MG\lib\site-packages\numba\core\decorators.py:253: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [3]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell_final'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin_final'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold_final'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosomes_final'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria_final'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus_final'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er_final'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi_final'

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack = np.zeros(shape = (5, 37440, 35168))
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in range(0, 5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack[i] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in range(len(coord_mito)):
    D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
    coord_mito.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_mito['volumen'] = coord_mito['mito_area'] * 200

In [ ]:
coord_mito

In [ ]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=26)

In [ ]:
mito_labels.max()

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in range(5):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_5_3d = mito_to_3d(mito_labels, coord_mito)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_5_3d.to_excel('features_mito_5_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack_30 = np.zeros(shape = (30, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(0,30)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack_30[i] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito_30 = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito_30['mito_area'] = mito_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito_30['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_mito_30))):
    D = max(coord_mito_30['max_row'][y] - coord_mito_30['min_row'][y], coord_mito_30['max_col'][y] -coord_mito_30['min_col'][y])
    coord_mito_30.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in pbar(range(30)):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
coord_mito_30['volumen'] = coord_mito_30['mito_area'] * 200

In [ ]:
mito_labels_30 = cc3d.connected_components(mito_stack_30, connectivity=26)

In [ ]:
start_time = time.time()
features_mito_30_3d = mito_to_3d(mito_labels_30, coord_mito_30)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_30_3d.to_excel('features_mito_30_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack_60 = np.zeros(shape = (30, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(30,60)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack_60[i-30] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito_60 = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito_60['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito_60['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in range(len(coord_mito_60)):
    D = max(coord_mito_60['max_row'][y] - coord_mito_60['min_row'][y], coord_mito_60['max_col'][y] -coord_mito_60['min_col'][y])
    coord_mito_60.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_mito_60['volumen'] = coord_mito_60['mito_area'] * 200

In [ ]:
mito_labels_60 = cc3d.connected_components(mito_stack_60, connectivity=26)

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in pbar(range(30)):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j + 30:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_60_3d = mito_to_3d(mito_labels_60, coord_mito_60)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_60_3d.to_excel('features_mito_60_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack_90 = np.zeros(shape = (30, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(60,90)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack_90[i-60] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito_90 = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito_90['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito_90['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in range(len(coord_mito_90)):
    D = max(coord_mito_90['max_row'][y] - coord_mito_90['min_row'][y], coord_mito_90['max_col'][y] -coord_mito_90['min_col'][y])
    coord_mito_90.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_mito_90['volumen'] = coord_mito_90['mito_area'] * 200

In [ ]:
mito_labels_90 = cc3d.connected_components(mito_stack_90, connectivity=26)

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in range(30):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j + 60:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_90_3d = mito_to_3d(mito_labels_90, coord_mito_90)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_90_3d.to_excel('features_mito_90_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack_120 = np.zeros(shape = (30, 37440, 35168))
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(90,120)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack_120[i-90] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito_120 = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito_120['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito_120['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_mito_120))):
    D = max(coord_mito_120['max_row'][y] - coord_mito_120['min_row'][y], coord_mito_120['max_col'][y] -coord_mito_120['min_col'][y])
    coord_mito_120.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_mito_120['volumen'] = coord_mito_120['mito_area'] * 200

In [ ]:
mito_labels_120 = cc3d.connected_components(mito_stack_120, connectivity=26)

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in pbar(range(30)):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j + 90:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_120_3d = mito_to_3d(mito_labels_120, coord_mito_120)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_120_3d.to_excel('features_mito_120_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack_155 = np.zeros(shape = (35, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(120,155)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack_155[i-120] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito_155 = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito_155['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito_155['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in range(len(coord_mito_155)):
    D = max(coord_mito_155['max_row'][y] - coord_mito_155['min_row'][y], coord_mito_155['max_col'][y] -coord_mito_155['min_col'][y])
    coord_mito_155.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_mito_155['volumen'] = coord_mito_155['mito_area'] * 200

In [ ]:
mito_labels_155 = cc3d.connected_components(mito_stack_155, connectivity=26)

In [ ]:
def mito_to_3d(mito_labels, coord_mito):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in pbar(range(30)):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j + 120:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_155_3d = mito_to_3d(mito_labels_155, coord_mito_155)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_155_3d.to_excel('features_mito_155_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_20 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(10,20)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_20[i-10] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_20 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_20['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_20['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_20))):
    D = max(coord_nuc_20['max_row'][y] - coord_nuc_20['min_row'][y], coord_nuc_20['max_col'][y] -coord_nuc_20['min_col'][y])
    coord_nuc_20.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_20['volumen'] = coord_nuc_20['nuc_area'] * 200

In [ ]:
nuc_labels_20 = cc3d.connected_components(nuc_stack_20, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 10:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_20_3d = nuc_to_3d(nuc_labels_20, coord_nuc_20)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_20_3d.to_excel('features_nuc_20_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_30 = np.zeros(shape = (10, 37440, 35168), dtype=np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(20,30)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_30[i-20] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_30 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_30['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_30['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_30))):
    D = max(coord_nuc_30['max_row'][y] - coord_nuc_30['min_row'][y], coord_nuc_30['max_col'][y] -coord_nuc_30['min_col'][y])
    coord_nuc_30.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_30['volumen'] = coord_nuc_30['nuc_area'] * 200

In [ ]:
nuc_labels_30 = cc3d.connected_components(nuc_stack_30, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 20:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_30_3d = nuc_to_3d(nuc_labels_30, coord_nuc_30)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_30_3d.to_excel('features_nuc_30_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_50 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(40,50)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_50[i-40] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_50 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_50['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_50['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_50))):
    D = max(coord_nuc_50['max_row'][y] - coord_nuc_50['min_row'][y], coord_nuc_50['max_col'][y] -coord_nuc_50['min_col'][y])
    coord_nuc_50.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_50['volumen'] = coord_nuc_50['nuc_area'] * 200

In [ ]:
nuc_labels_50 = cc3d.connected_components(nuc_stack_50, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in range(10):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 40:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_50_3d = nuc_to_3d(nuc_labels_50, coord_nuc_50)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_50_3d.to_excel('features_nuc_50_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_80 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(70,80)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_80[i-70] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_80 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_80['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_80['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_80))):
    D = max(coord_nuc_80['max_row'][y] - coord_nuc_80['min_row'][y], coord_nuc_80['max_col'][y] -coord_nuc_80['min_col'][y])
    coord_nuc_80.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_80['volumen'] = coord_nuc_80['nuc_area'] * 200

In [ ]:
nuc_labels_80 = cc3d.connected_components(nuc_stack_80, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in range(10):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 70:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_80_3d = nuc_to_3d(nuc_labels_80, coord_nuc_80)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_80_3d.to_excel('features_nuc_80_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_90 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(80,90)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_90[i-80] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_90 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_90['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_90['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_90))):
    D = max(coord_nuc_90['max_row'][y] - coord_nuc_90['min_row'][y], coord_nuc_90['max_col'][y] -coord_nuc_90['min_col'][y])
    coord_nuc_90.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_90['volumen'] = coord_nuc_90['nuc_area'] * 200

In [ ]:
nuc_labels_90 = cc3d.connected_components(nuc_stack_90, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 80:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_90_3d = nuc_to_3d(nuc_labels_90, coord_nuc_90)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_90_3d.to_excel('features_nuc_90_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_100 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(90,100)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_100[i-90] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_100 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_100['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_100['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_100))):
    D = max(coord_nuc_100['max_row'][y] - coord_nuc_100['min_row'][y], coord_nuc_100['max_col'][y] -coord_nuc_100['min_col'][y])
    coord_nuc_100.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_100['volumen'] = coord_nuc_100['nuc_area'] * 200

In [ ]:
nuc_labels_100 = cc3d.connected_components(nuc_stack_100, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 90:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_100_3d = nuc_to_3d(nuc_labels_100, coord_nuc_100)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_100_3d.to_excel('features_nuc_100_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_110 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(100,110)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_110[i-100] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_110 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_110['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_110['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_110))):
    D = max(coord_nuc_110['max_row'][y] - coord_nuc_110['min_row'][y], coord_nuc_110['max_col'][y] -coord_nuc_110['min_col'][y])
    coord_nuc_110.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_110['volumen'] = coord_nuc_110['nuc_area'] * 200

In [ ]:
nuc_labels_110 = cc3d.connected_components(nuc_stack_110, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 100:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_110_3d = nuc_to_3d(nuc_labels_110, coord_nuc_110)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_110_3d.to_excel('features_nuc_110_3d_v1.xlsx')

In [12]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_120 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(110,120)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_120[i-110] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_120 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_120['nuc_area'] = nuc_area_values_combined

100% (10 of 10) |########################| Elapsed Time: 0:10:28 Time:  0:10:28


In [13]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_120['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_120))):
    D = max(coord_nuc_120['max_row'][y] - coord_nuc_120['min_row'][y], coord_nuc_120['max_col'][y] -coord_nuc_120['min_col'][y])
    coord_nuc_120.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

100% (17 of 17) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Elapsed time:  0.021199703216552734


In [14]:
coord_nuc_120['volumen'] = coord_nuc_120['nuc_area'] * 200

In [15]:
nuc_labels_120 = cc3d.connected_components(nuc_stack_120, connectivity=26)

In [16]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 110:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [17]:
start_time = time.time()
features_nuc_120_3d = nuc_to_3d(nuc_labels_120, coord_nuc_120)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

 30% (3 of 10) |#######                  | Elapsed Time: 0:00:01 ETA:   0:00:03C:\Users\Gast-User\AppData\Local\Temp\ipykernel_17300\2042944888.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_nuc['nuc_3d'][n] = i
100% (10 of 10) |########################| Elapsed Time: 0:02:14 Time:  0:02:14


Elapsed time:  134.91941213607788


In [18]:
features_nuc_120_3d.to_excel('features_nuc_120_3d_v1.xlsx')

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_140 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(130,140)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_140[i-130] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_140 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_140['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_140['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_140))):
    D = max(coord_nuc_140['max_row'][y] - coord_nuc_140['min_row'][y], coord_nuc_140['max_col'][y] -coord_nuc_140['min_col'][y])
    coord_nuc_140.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
coord_nuc_140['volumen'] = coord_nuc_140['nuc_area'] * 200

In [ ]:
nuc_labels_140 = cc3d.connected_components(nuc_stack_140, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(10)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 130:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_140_3d = nuc_to_3d(nuc_labels_140, coord_nuc_140)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_140_3d.to_excel('features_nuc_140_3d_v1.xlsx')

In [4]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_155 = np.zeros(shape = (15, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(140,155)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_155[i-140] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_155 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_155['nuc_area'] = nuc_area_values_combined

100% (15 of 15) |########################| Elapsed Time: 0:16:08 Time:  0:16:08


In [5]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_155['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_155))):
    D = max(coord_nuc_155['max_row'][y] - coord_nuc_155['min_row'][y], coord_nuc_155['max_col'][y] -coord_nuc_155['min_col'][y])
    coord_nuc_155.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

100% (40 of 40) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Elapsed time:  0.01814866065979004


In [7]:
coord_nuc_155['volumen'] = coord_nuc_155['nuc_area'] * 200

In [8]:
nuc_labels_155 = cc3d.connected_components(nuc_stack_155, connectivity=26)

In [9]:
def nuc_to_3d(nuc_labels, coord_nuc):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(15)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + 140:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [10]:
start_time = time.time()
features_nuc_155_3d = nuc_to_3d(nuc_labels_155, coord_nuc_155)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

  0% (0 of 15) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\Gast-User\AppData\Local\Temp\ipykernel_17300\793204407.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_nuc['nuc_3d'][n] = i
100% (15 of 15) |########################| Elapsed Time: 0:10:19 Time:  0:10:19


Elapsed time:  619.8422141075134


In [11]:
features_nuc_155_3d.to_excel('features_nuc_155_3d_v1.xlsx')

In [ ]:
mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))

In [ ]:
mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)

###  connectivity=26

In [ ]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

### connectivity=18

In [ ]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=18)

In [ ]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

### connectivity=6

In [ ]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=6)

In [ ]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

In [ ]:
coord_mito = np.where(mito_labels[0] == 0)

In [ ]:
coord_mito

# Nuclear calculations

In [ ]:
start_time = time.time()
nuc_stack = np.zeros(shape = (5, 37440, 35168))
all_nuc_data = []
for i in range(0,5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    all_nuc_data.extend(mito_coord)
    nuc_stack[i] = nuc_label
coord_nuc = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
#       coord_mito['sl_num'][r] = i
unique_sl_nums = coord_mito['sl_num'].unique()
for sl_num in unique_sl_nums:
    mito_list = np.unique(coord_mito[coord_mito['sl_num'] == sl_num]['sl_num'])
    mito_area_values = get_area(mito_label, mito_list)
    coord_mito.loc[coord_mito['sl_num'] == sl_num, 'mito_area'] = mito_area_values
coord_mito['mito_large'] = 0
for y in range(len(coord_mito)):
    D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
    coord_mito['mito_large'][y] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)